In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score, precision_recall_fscore_support
from tqdm.notebook import tqdm
import timm
import os
import random
import csv
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

Using device: cuda
PyTorch version: 2.6.0+cu118


In [2]:
CSV_PATH = r"F:\Soil_Labeled_Data\labels.csv"
#CHECKPOINT_PATH = r"C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_model_final_mu_23.0.pth"
CHECKPOINT_PATH = r"C:\Users\PC\soilNet\checkpoints_VicReg\vicreg_model_final_mu_30.0.pth"
BEST_MODEL_SAVE_PATH = r"C:\Users\PC\soilNet\NEW\Best_finetuned_VicReg_mu_30.pth"
METRICS_CSV_PATH = r"C:\Users\PC\soilNet\NEW\training_metrics_finetuned_VicReg_mu_30.csv"

# Hyperparameters
BATCH_SIZE = 16
NUM_EPOCHS = 100
LEARNING_RATE = 1e-4

# Create directories
os.makedirs(os.path.dirname(BEST_MODEL_SAVE_PATH), exist_ok=True)
os.makedirs(os.path.dirname(METRICS_CSV_PATH), exist_ok=True)

In [3]:
df = pd.read_csv(CSV_PATH)
print(f"Total labeled images before check: {len(df)}")

required_columns = ['path', 'SM_0', 'SM_20', 'light_value', 'moisture_class']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"DataFrame missing columns: {missing_columns}")

# Image fallback & augmentation
fallback_dir = r"F:\Soil_Labeled_Data\augmented_fallback"
os.makedirs(fallback_dir, exist_ok=True)
replaced_count = 0
augment = transforms.ColorJitter(brightness=0.2, contrast=0.2)

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Checking images"):
    path = row["path"]
    try:
        img = Image.open(path).convert("RGB")
        img.verify()
    except Exception as e:
        print(f"Image error: {path} -> {e}")
        folder = os.path.dirname(path)
        all_images = [f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".png"))]
        good_images = [f for f in all_images if f != os.path.basename(path)]
        if not good_images:
            print(f"No replacement in: {folder}")
            df = df.drop(idx)
            continue
        candidate = random.choice(good_images)
        candidate_path = os.path.join(folder, candidate)
        try:
            img = Image.open(candidate_path).convert("RGB")
            img_aug = augment(img)
            new_filename = f"aug_{os.path.basename(path)}"
            new_path = os.path.join(fallback_dir, new_filename)
            img_aug.save(new_path)
            df.at[idx, "path"] = new_path
            replaced_count += 1
            print(f"Replaced with: {new_path}")
        except Exception as e2:
            print(f"Replacement failed: {candidate_path} -> {e2}")
            df = df.drop(idx)
print(f"Replaced {replaced_count} invalid images. Final dataset size: {len(df)}")

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Total labeled images before check: 2057


Checking images:   0%|          | 0/2057 [00:00<?, ?it/s]

Image error: F:\Soil_Labeled_Data\M_10_20\L_60_70\alluvial\IMG_0683.JPG -> [Errno 2] No such file or directory: 'F:\\Soil_Labeled_Data\\M_10_20\\L_60_70\\alluvial\\IMG_0683.JPG'
Replaced with: F:\Soil_Labeled_Data\augmented_fallback\aug_IMG_0683.JPG
Image error: F:\Soil_Labeled_Data\M_10_20\L_70_80\alluvial\IMG_0703.JPG -> [Errno 2] No such file or directory: 'F:\\Soil_Labeled_Data\\M_10_20\\L_70_80\\alluvial\\IMG_0703.JPG'
Replaced with: F:\Soil_Labeled_Data\augmented_fallback\aug_IMG_0703.JPG
Image error: F:\Soil_Labeled_Data\M_10_20\L_80_90\alluvial\IMG_0746.JPG -> [Errno 2] No such file or directory: 'F:\\Soil_Labeled_Data\\M_10_20\\L_80_90\\alluvial\\IMG_0746.JPG'
Replaced with: F:\Soil_Labeled_Data\augmented_fallback\aug_IMG_0746.JPG
Image error: F:\Soil_Labeled_Data\M_10_20\L_80_90\alluvial\IMG_0753.JPG -> [Errno 2] No such file or directory: 'F:\\Soil_Labeled_Data\\M_10_20\\L_80_90\\alluvial\\IMG_0753.JPG'
Replaced with: F:\Soil_Labeled_Data\augmented_fallback\aug_IMG_0753.JPG


In [4]:
class SoilDualTaskDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["path"]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        light = torch.tensor([row["light_value"] / 100.0], dtype=torch.float32)
        regression_target = torch.tensor([row["SM_0"] / 100.0, row["SM_20"] / 100.0], dtype=torch.float32)
        class_target = torch.tensor(row["moisture_class"], dtype=torch.long)
        return {
            "image": img,
            "light": light,
            "regression_target": regression_target,
            "class_target": class_target
        }

dataset = SoilDualTaskDataset(df, transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, 
                        pin_memory=True if device.type == "cuda" else False, num_workers=0)

In [5]:
class SoilNetDualHead(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.initial_conv = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        mnv2 = timm.create_model("mobilenetv2_100.ra_in1k", pretrained=True)
        self.mnv2_block1 = nn.Sequential(*list(mnv2.blocks.children())[:3])
        self.channel_adapter = nn.Conv2d(32, 16, kernel_size=1, bias=False)
        self.mobilevit_full = timm.create_model("mobilevitv2_050", pretrained=True)
        self.mobilevit_encoder = self.mobilevit_full.stages
        self.mvit_to_mnv2 = nn.Conv2d(256, 32, kernel_size=1, bias=False)
        self.mnv2_block2 = nn.Sequential(*list(mnv2.blocks.children())[3:7])
        self.final_conv = nn.Conv2d(320, 1280, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.light_dense = nn.Sequential(nn.Linear(1, 32), nn.ReLU(inplace=True))
        self.reg_head = nn.Sequential(
            nn.Linear(1280 + 32, 128), nn.ReLU(inplace=True), nn.Linear(128, 2)
        )
        self.cls_head = nn.Sequential(
            nn.Linear(1280 + 32, 128), nn.ReLU(inplace=True), nn.Linear(128, num_classes)
        )

    def forward(self, x_img, x_light):
        x = self.initial_conv(x_img)
        x = self.mnv2_block1(x)
        x = self.channel_adapter(x)
        x = self.mobilevit_encoder(x)
        x = self.mvit_to_mnv2(x)
        x = self.mnv2_block2(x)
        x = self.final_conv(x)
        x = self.pool(x)
        x_img_feat = torch.flatten(x, 1)
        x_light_feat = self.light_dense(x_light)
        x_concat = torch.cat([x_img_feat, x_light_feat], dim=1)
        reg_out = self.reg_head(x_concat)
        cls_out = self.cls_head(x_concat)
        return reg_out, cls_out

num_classes = len(df["moisture_class"].unique())
model = SoilNetDualHead(num_classes=num_classes).to(device)

# Load pretrained
try:
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=device, weights_only=False)
    model_state_dict = checkpoint['model_state_dict'] if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint else checkpoint
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in model_state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict, strict=False)
    print(f"Loaded pretrained weights from {CHECKPOINT_PATH}")
except Exception as e:
    print(f"Error loading checkpoint: {e}. Using random init.")

Loaded pretrained weights from C:\Users\PC\soilNet\checkpoints_VicReg\vicreg_model_final_mu_30.0.pth


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
reg_criterion = nn.MSELoss()
cls_criterion = nn.CrossEntropyLoss()

# Metrics CSV header
metrics_header = [
    "Epoch", "Total_Loss", "Reg_Loss", "Cls_Loss",
    "RMSE_SM0", "MAE_SM0", "ME_SM0",
    "RMSE_SM20", "MAE_SM20", "ME_SM20",
    "Accuracy", "F1_Score", "Precision", "Recall"
]
with open(METRICS_CSV_PATH, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(metrics_header)

# Best metric tracking
best_metric = float('inf')
best_epoch = -1

In [7]:
print("Starting training...")
for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    model.train()
    total_loss, total_reg_loss, total_cls_loss = 0, 0, 0

    for batch in tqdm(dataloader, desc="Training", leave=False):
        x_img = batch["image"].to(device)
        x_light = batch["light"].to(device)
        y_reg = batch["regression_target"].to(device)
        y_cls = batch["class_target"].to(device)

        optimizer.zero_grad()
        pred_reg, pred_cls = model(x_img, x_light)
        loss_reg = reg_criterion(pred_reg, y_reg)
        loss_cls = cls_criterion(pred_cls, y_cls)
        loss = loss_reg + loss_cls
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_reg_loss += loss_reg.item()
        total_cls_loss += loss_cls.item()

    # Average losses
    avg_loss = total_loss / len(dataloader)
    avg_reg_loss = total_reg_loss / len(dataloader)
    avg_cls_loss = total_cls_loss / len(dataloader)

    # ========================= EVALUATION =========================
    model.eval()
    y_true_reg, y_pred_reg = [], []
    y_true_cls, y_pred_cls = [], []

    with torch.no_grad():
        for batch in dataloader:
            x_img = batch["image"].to(device)
            x_light = batch["light"].to(device)
            y_reg = batch["regression_target"].to(device)
            y_cls = batch["class_target"].to(device)
            pred_reg, pred_cls = model(x_img, x_light)
            y_true_reg.extend(y_reg.cpu().numpy())
            y_pred_reg.extend(pred_reg.cpu().numpy())
            y_true_cls.extend(y_cls.cpu().numpy())
            y_pred_cls.extend(pred_cls.argmax(dim=1).cpu().numpy())

    # Convert back to original scale
    y_true_reg = np.array(y_true_reg) * 100
    y_pred_reg = np.array(y_pred_reg) * 100
    y_true_cls = np.array(y_true_cls)
    y_pred_cls = np.array(y_pred_cls)

    # Regression metrics
    rmse_sm0 = mean_squared_error(y_true_reg[:, 0], y_pred_reg[:, 0]) ** 0.5
    mae_sm0 = mean_absolute_error(y_true_reg[:, 0], y_pred_reg[:, 0])
    me_sm0 = np.mean(y_pred_reg[:, 0] - y_true_reg[:, 0])

    rmse_sm20 = mean_squared_error(y_true_reg[:, 1], y_pred_reg[:, 1]) ** 0.5
    mae_sm20 = mean_absolute_error(y_true_reg[:, 1], y_pred_reg[:, 1])
    me_sm20 = np.mean(y_pred_reg[:, 1] - y_true_reg[:, 1])

    # Classification metrics
    acc = accuracy_score(y_true_cls, y_pred_cls)
    f1 = f1_score(y_true_cls, y_pred_cls, average='weighted')
    precision, recall, _, _ = precision_recall_fscore_support(y_true_cls, y_pred_cls, average='weighted')

    # Composite score for best model (lower is better)
    composite_score = (rmse_sm0 + rmse_sm20) / 2 + (1 - acc)

    # Save best model only
    if composite_score < best_metric:
        best_metric = composite_score
        best_epoch = epoch + 1
        torch.save(model.state_dict(), BEST_MODEL_SAVE_PATH)
        print(f"New best model saved! Score: {composite_score:.6f}")

    # Log to CSV
    row = [
        epoch + 1, f"{avg_loss:.6f}", f"{avg_reg_loss:.6f}", f"{avg_cls_loss:.6f}",
        f"{rmse_sm0:.6f}", f"{mae_sm0:.6f}", f"{me_sm0:.6f}",
        f"{rmse_sm20:.6f}", f"{mae_sm20:.6f}", f"{me_sm20:.6f}",
        f"{acc:.6f}", f"{f1:.6f}", f"{precision:.6f}", f"{recall:.6f}"
    ]
    with open(METRICS_CSV_PATH, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)

    # Print
    print(f"Loss → Total: {avg_loss:.4f}, Reg: {avg_reg_loss:.4f}, Cls: {avg_cls_loss:.4f}")
    print(f"SM_0  → RMSE: {rmse_sm0:.6f}, MAE: {mae_sm0:.6f}, ME: {me_sm0:.6f}")
    print(f"SM_20 → RMSE: {rmse_sm20:.6f}, MAE: {mae_sm20:.6f}, ME: {me_sm20:.6f}")
    print(f"Cls   → Acc: {acc:.6f}, F1: {f1:.6f}, Prec: {precision:.6f}, Rec: {recall:.6f}")

print(f"\nTraining completed! Best model (epoch {best_epoch}) saved at:\n{BEST_MODEL_SAVE_PATH}")
print(f"Metrics logged to:\n{METRICS_CSV_PATH}")

Starting training...

Epoch 1/100


Training:   0%|          | 0/129 [00:00<?, ?it/s]

New best model saved! Score: 20.758783
Loss → Total: 2.1714, Reg: 0.0627, Cls: 2.1087
SM_0  → RMSE: 19.783296, MAE: 15.846496, ME: 4.161200
SM_20 → RMSE: 20.382787, MAE: 16.233181, ME: 2.496105
Cls   → Acc: 0.324259, F1: 0.314183, Prec: 0.327117, Rec: 0.324259

Epoch 2/100


Training:   0%|          | 0/129 [00:00<?, ?it/s]

New best model saved! Score: 16.652646
Loss → Total: 1.8709, Reg: 0.0404, Cls: 1.8305
SM_0  → RMSE: 16.034312, MAE: 12.615541, ME: 0.908582
SM_20 → RMSE: 16.042978, MAE: 12.708213, ME: 0.176238
Cls   → Acc: 0.385999, F1: 0.376042, Prec: 0.387904, Rec: 0.385999

Epoch 3/100


...

Loss → Total: 0.2976, Reg: 0.0057, Cls: 0.2919
SM_0  → RMSE: 5.986251, MAE: 4.539366, ME: -1.283935
SM_20 → RMSE: 6.115054, MAE: 4.758543, ME: -1.366144
Cls   → Acc: 0.958192, F1: 0.958637, Prec: 0.960271, Rec: 0.958192

Epoch 99/100


Training:   0%|          | 0/129 [00:00<?, ?it/s]

Loss → Total: 0.2781, Reg: 0.0052, Cls: 0.2729
SM_0  → RMSE: 5.731731, MAE: 4.317306, ME: -0.389968
SM_20 → RMSE: 5.918235, MAE: 4.512618, ME: -0.187625
Cls   → Acc: 0.953330, F1: 0.954132, Prec: 0.958662, Rec: 0.953330

Epoch 100/100


Training:   0%|          | 0/129 [00:00<?, ?it/s]

New best model saved! Score: 5.736288
Loss → Total: 0.2502, Reg: 0.0047, Cls: 0.2455
SM_0  → RMSE: 5.573007, MAE: 4.179292, ME: -0.527088
SM_20 → RMSE: 5.813034, MAE: 4.416228, ME: -0.182294
Cls   → Acc: 0.956733, F1: 0.956903, Prec: 0.957569, Rec: 0.956733

Training completed! Best model (epoch 100) saved at:
C:\Users\PC\soilNet\NEW\Best_finetuned_VicReg_mu_30.pth
Metrics logged to:
C:\Users\PC\soilNet\NEW\training_metrics_finetuned_VicReg_mu_30.csv
